In [ ]:
# Associadores:
    
Ex: mais comum, sistemas de recomendação. qm compra A, geralmente compra B.
baseado no que clientes passados parecidos fizeram. 

In [ ]:
# Conceitos

A=>B

métricas de associação:
- Suporte: número de transações (ocorrências) que contém os elementos comparados. Não adianta os produtos 
    aparecerem com frequência juntos, se raramente são comprados!
- Confiança: indica a proporção de vezes que, em uma transação contendo o elemento A, também tem B. Não 
    adianta ser muito comprado se dificilmente eles aparecem juntos nas compras (ocorrência). 
- Lift: o quanto aumenta a frequência de B com a ocorrência de A. confiança/suporte

# Exemplos:

- 1 <br>
![title](ex_FPGrowth.png) 
<br>
- 2 <br>
![title](ex2_FPGrowth.png) 
<br>
- 3 <br>
![title](ex3_FPGrowth.png) 

In [ ]:
# Mecanismo:
- induz árvores com transações, e busca sobreposição destas árvores onde os itens são frequentes. 

In [ ]:
# Hiperparâmetros:
- minConfidence: confiança mínima para gerar regra (padrão: 0.8)
- minSupport: suporte mínimo para gerar a regra. (padrão:0.3)

In [2]:
import findspark, pyspark
from pyspark.sql import SparkSession
findspark.init()
spark = SparkSession.builder.appName("hierarchical").getOrCreate()

In [1]:
from pyspark.ml.fpm import FPGrowth

In [6]:
from pyspark.sql.functions import split
transacoes = spark.read.text("Transacoes.txt").select(split("value","\s+").alias("transacoes"))
transacoes.show(truncate=False)

+-------------------------+
|transacoes               |
+-------------------------+
|[Cerveja, Pizza, Sorvete]|
|[Pizza, Sorvete]         |
|[Cerveja, Pizza]         |
|[Cerveja, Pizza, Sorvete]|
|[Cerveja, Pizza]         |
|[Pizza]                  |
+-------------------------+



In [9]:
fp = FPGrowth(itemsCol="transacoes", minSupport=0.5, minConfidence=0.5)
modelo = fp.fit(transacoes)
modelo.freqItemsets.show()

Exception ignored in: <function JavaWrapper.__del__ at 0x00000155F6BB0940>
Traceback (most recent call last):
  File "C:\Users\felip\anaconda3\lib\site-packages\pyspark\ml\wrapper.py", line 53, in __del__
    if SparkContext._active_spark_context and self._java_obj is not None:
AttributeError: 'FPGrowth' object has no attribute '_java_obj'


+----------------+----+
|           items|freq|
+----------------+----+
|       [Sorvete]|   3|
|[Sorvete, Pizza]|   3|
|       [Cerveja]|   4|
|[Cerveja, Pizza]|   4|
|         [Pizza]|   6|
+----------------+----+



In [10]:
modelo.associationRules.show()

+----------+----------+------------------+----+------------------+
|antecedent|consequent|        confidence|lift|           support|
+----------+----------+------------------+----+------------------+
| [Cerveja]|   [Pizza]|               1.0| 1.0|0.6666666666666666|
| [Sorvete]|   [Pizza]|               1.0| 1.0|               0.5|
|   [Pizza]| [Sorvete]|               0.5| 1.0|               0.5|
|   [Pizza]| [Cerveja]|0.6666666666666666| 1.0|0.6666666666666666|
+----------+----------+------------------+----+------------------+



In [11]:
# Baseado no que a pessoa comprou, prevê o que ela pode levar:
modelo.transform(transacoes).show(truncate=False)

+-------------------------+------------------+
|transacoes               |prediction        |
+-------------------------+------------------+
|[Cerveja, Pizza, Sorvete]|[]                |
|[Pizza, Sorvete]         |[Cerveja]         |
|[Cerveja, Pizza]         |[Sorvete]         |
|[Cerveja, Pizza, Sorvete]|[]                |
|[Cerveja, Pizza]         |[Sorvete]         |
|[Pizza]                  |[Sorvete, Cerveja]|
+-------------------------+------------------+

